In [116]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import seaborn as sns
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import  mean
from sklearn.ensemble import BaggingClassifier

import warnings
warnings.filterwarnings('ignore')


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


In [117]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [118]:
df_train_set = pd.read_csv('/kaggle/input/south-african-language-identification-hack-2022/train_set.csv')
df_test_set = pd.read_csv('/kaggle/input/south-african-language-identification-hack-2022/test_set.csv')

In [119]:
df_train_set.head(3)

In [120]:
df_test_set.head()

In [121]:
def clean_data(text_data): 
    text_data= re.sub("[^a-zA-Z0-9 ]", "", text_data) #Extracting Remaining text from  
    text_data = re.sub(r'[,!?;-]+', '.', text_data) #Removing any remaining non-characters
    text_data = text_data.replace('[^a-zA-Z#]','')
    text_data = text_data.lower() # Change words to lower case
    text_data= text_data.lstrip('\'"') # Remove extra white space
    return text_data

In [122]:
df_train_set['cleaned_text'] = df_train_set['text'].apply(clean_data)
df_test_set['cleaned_text'] = df_test_set['text'].apply(clean_data)

In [123]:
# remove numbers
#df_train_set['cleaned_text'] = df_train_set['cleaned_text'].str.rstrip(string.digits)
#df_test_set['cleaned_text'] = df_test_set['cleaned_text'].str.rstrip(string.digits)
df_train_set['cleaned_text'] = df_train_set['cleaned_text'].str.replace('\d+', '')
df_test_set['cleaned_text'] = df_test_set['cleaned_text'].str.replace('\d+', '')

In [124]:
df_train_set.head(20)

In [125]:
df_test_set.head(20)

Check all the languanges available on our data set

In [126]:
languages = list(df_train_set.lang_id.unique())
print(languages)

language distribution

In [127]:
lang_distr = pd.DataFrame(list(df_train_set['lang_id'].value_counts()),columns=['Count'])
lang_distr

In [128]:
df_train_set['text'].nunique()

In [129]:
df_train_set.shape

It is quite clear that we have a lot of duplicates in our data set, We are now going to drop the duplicated data

In [130]:
df_test_set.shape

In [131]:
df_test_set['text'].nunique()

In [132]:
# function to drop duplicates
df_train_set.drop_duplicates(keep=False, inplace=True)
df_test_set.drop_duplicates(keep=False, inplace=True)

In [133]:
df_train_set.shape

In [134]:
df_test_set.shape

In [135]:
# Check for missing values
df_train_set.isnull().sum()

In [136]:
# Check for missing values
df_test_set.isnull().sum()

In [137]:
#code to encode values to numeric
#inc = LabelEncoder()
#label = inc.fit_transform(df_train_set['Purchased'])

# Tokenisation

In [138]:
# Creating token for the clean language
df_train_set['text_tokens'] = df_train_set['cleaned_text'].apply(lambda x: word_tokenize(x))
df_test_set['text_tokens'] = df_test_set['cleaned_text'].apply(lambda x: word_tokenize(x))

In [139]:
df_train_set.head()

# Lammatization

In [140]:
lemmatizer = WordNetLemmatizer()
def text_lamma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]

In [141]:

df_train_set['lem_text'] = df_train_set['text_tokens'].apply(text_lamma, args=(lemmatizer, ))
df_test_set['lem_text'] = df_test_set['text_tokens'].apply(text_lamma, args=(lemmatizer, ))

In [142]:
df_train_set.head(2)

Feature extraction

In [143]:
X= df_train_set['cleaned_text']
X_val =df_test_set['cleaned_text']
y = df_train_set['lang_id']

In [144]:
vectorizer = TfidfVectorizer(sublinear_tf=True)
X = vectorizer.fit_transform(X)
X_val = vectorizer.transform(X_val)

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 42)

In [146]:
#Creating a baseline prediction to be used to access model performance
acc_baseline = y_train.value_counts(normalize=True).max()
print("Baseline Accuracy:", round(acc_baseline, 2))

In [147]:
# Training model using Naive bayes classifier
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

y_pred=nb_model.predict(X_test)
print(f'Trainging set prediction performance is {nb_model.score(X_train, y_train) }')
print(f'Testing set prediction performance is {nb_model.score(X_test, y_test) }')

In [148]:
#Logistic regression model for training 
log_model = LogisticRegression(multi_class='ovr', solver='liblinear')
log_model.fit(X_train, y_train)

print(f'Trainging set prediction performance is {log_model.score(X_train, y_train)}')
print(f'Testing set prediction performance is {log_model.score(X_test,y_test) }')

In [149]:
#random forest model for model training 
rf_model = RandomForestClassifier(min_samples_leaf = 2,n_estimators=400)
rf_model.fit(X_train,y_train)

print(f'Trainging set prediction performance is {rf_model.score(X_train, y_train) }')
print(f'Testing set prediction performance is {rf_model.score(X_test, y_test) }')

In [150]:
#Create an instance
classifier = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42)
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)

In [151]:
classifier.score(X_test, y_test) 

In [152]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='weighted')

In [153]:

print(metrics.classification_report(y_test, preds))

In [154]:
prediction = log_model.predict(X_val)

In [155]:
prediction[:10]

In [156]:
df_test_set['lang_id'] = prediction

In [157]:
df_test_set.rename(columns={'pred':'lang_id'},inplace=True)

In [158]:
submission = df_test_set[['index','lang_id']]

In [159]:
submission.head()

In [160]:
submission = submission.set_index('lang_id')

In [161]:
submission.to_csv('submission.csv')